In [1]:

from pathlib import Path
import pandas as pd
from itertools import product
from chan import detect_segments, analyze_segments_characteristics

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
DATA_BI = ROOT / "chan_data/bi"
SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m", "15m", "30m", "1h", "4h"]

def run_segment_sweep(label, *, min_bars, min_rel_move, break_validation=False, break_mode="last_block_extreme"):
    rows = []
    for s in SYMBOLS:
        for tf in TFS:
            p = DATA_BI / s / f"{tf}_bi.csv"
            if not p.exists():
                continue
            df_bi = pd.read_csv(p, parse_dates=["timestamp_start", "timestamp_end"])
            seg = detect_segments(
                df_bi,
                min_bars=min_bars,
                min_rel_move=min_rel_move,
                enable_break_validation=break_validation,
                break_mode=break_mode,
                merge_same_dir=True
            )
            stat = analyze_segments_characteristics(seg)
            if stat:
                rows.append({
                    "version": label, "symbol": s, "tf": tf,
                    "total_segments": stat["total_segments"],
                    "avg_bi_count": stat["avg_bi_count"],
                    "avg_time_hours": stat["avg_time_hours"],
                    "avg_price_range": stat["avg_price_range"],
                    "up_down_ratio": stat["up_down_ratio"],
                    "avg_overall_strength": stat["avg_overall_strength"],
                })
    return pd.DataFrame(rows)


df_fast = run_segment_sweep("fast", min_bars=2, min_rel_move=0.001, break_validation=False, break_mode="last_block_extreme")

df_slow = run_segment_sweep("slow", min_bars=3, min_rel_move=0.005, break_validation=False, break_mode="last_block_extreme")


both = pd.concat([df_fast, df_slow], ignore_index=True)
pivot = both.pivot_table(
    index=["symbol","tf"],
    columns="version",
    values=["total_segments","avg_bi_count","avg_overall_strength","up_down_ratio"],
    aggfunc="mean"
)


diff = pivot.copy()
for col in diff.columns.levels[0]:
    if ("fast" in pivot[col].columns) and ("slow" in pivot[col].columns):
        diff[(col,"delta(slow-fast)")] = pivot[(col,"slow")] - pivot[(col,"fast")]


print("== Comparison of the total number of paragraphs ==")
print("Fast total number of sections：", int(df_fast["total_segments"].sum()))
print("Slow total number of segments：", int(df_slow["total_segments"].sum()))
print("Sparse proportion slow/fast：{:.2%}".format(df_slow["total_segments"].sum() / max(1, df_fast["total_segments"].sum())))

display(pivot.round(3))
display(diff.round(3))


== Comparison of the total number of paragraphs ==
Fast total number of sections： 1318
Slow total number of segments： 494
Sparse proportion slow/fast：37.48%


avg_bi_count        avg_overall_strength        total_segments  \
version             fast   slow                 fast   slow           fast   
symbol  tf                                                                   
BTCUSDT 15m        2.672  3.560                0.518  0.523          116.0   
        1h         2.562  3.500                0.523  0.530           64.0   
        30m        2.524  3.486                0.519  0.524          105.0   
        4h         2.235  3.333                0.539  0.555           17.0   
        5m         2.643  3.674                0.518  0.524          112.0   
ETHUSDT 15m        2.535  3.457                0.523  0.532          256.0   
        1h         2.551  3.593                0.542  0.558           78.0   
        30m        2.423  3.250                0.531  0.543          142.0   
        4h         2.400  3.333                0.593  0.641           20.0   
        5m         2.635  3.599                0.518  0.524          408.0   

                   up_down_ratio         
version       slow          fast   slow  
symbol  tf                               
BTCUSDT 15m   50.0         1.071  1.174  
        1h    24.0         1.133  2.429  
        30m   37.0         0.944  1.176  
        4h     3.0         0.889  2.000  
        5m    43.0         1.036  1.529  
ETHUSDT 15m   94.0         1.098  1.186  
        1h    27.0         1.438  1.700  
        30m   48.0         1.185  2.692  
        4h     6.0         1.000  2.000  
        5m   162.0         1.050  1.189

avg_bi_count        avg_overall_strength        total_segments  \
version             fast   slow                 fast   slow           fast   
symbol  tf                                                                   
BTCUSDT 15m        2.672  3.560                0.518  0.523          116.0   
        1h         2.562  3.500                0.523  0.530           64.0   
        30m        2.524  3.486                0.519  0.524          105.0   
        4h         2.235  3.333                0.539  0.555           17.0   
        5m         2.643  3.674                0.518  0.524          112.0   
ETHUSDT 15m        2.535  3.457                0.523  0.532          256.0   
        1h         2.551  3.593                0.542  0.558           78.0   
        30m        2.423  3.250                0.531  0.543          142.0   
        4h         2.400  3.333                0.593  0.641           20.0   
        5m         2.635  3.599                0.518  0.524          408.0   

                   up_down_ratio            avg_bi_count avg_overall_strength  \
version       slow          fast   slow delta(slow-fast)     delta(slow-fast)   
symbol  tf                                                                      
BTCUSDT 15m   50.0         1.071  1.174            0.888                0.004   
        1h    24.0         1.133  2.429            0.938                0.007   
        30m   37.0         0.944  1.176            0.963                0.006   
        4h     3.0         0.889  2.000            1.098                0.016   
        5m    43.0         1.036  1.529            1.032                0.007   
ETHUSDT 15m   94.0         1.098  1.186            0.922                0.009   
        1h    27.0         1.438  1.700            1.041                0.016   
        30m   48.0         1.185  2.692            0.827                0.013   
        4h     6.0         1.000  2.000            0.933                0.047   
        5m   162.0         1.050  1.189            0.964                0.006   

              total_segments    up_down_ratio  
version     delta(slow-fast) delta(slow-fast)  
symbol  tf                                     
BTCUSDT 15m            -66.0            0.102  
        1h             -40.0            1.295  
        30m            -68.0            0.232  
        4h             -14.0            1.111  
        5m             -69.0            0.493  
ETHUSDT 15m           -162.0            0.088  
        1h             -51.0            0.262  
        30m            -94.0            1.508  
        4h             -14.0            1.000  
        5m            -246.0            0.139

In [21]:

from pathlib import Path
import pandas as pd
from chan import detect_segments

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
DATA_BI = ROOT / "chan_data" / "bi"
OUT_DIR = ROOT / "chan_data" / "segments" / "slow"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m","15m","30m","1h","4h"]

SEG_PARAMS_SLOW = dict(
    min_bars=3,
    min_rel_move=0.005,
    enable_break_validation=False,
    break_mode="last_block_extreme",
    merge_same_dir=True,
    enable_quality_markers=True
)

for s in SYMBOLS:
    (OUT_DIR / s).mkdir(exist_ok=True, parents=True)
    for tf in TFS:
        p_bi = DATA_BI / s / f"{tf}_bi.csv"
        if not p_bi.exists():
            print(f"[skip] missing {p_bi}")
            continue
        bi = pd.read_csv(p_bi, parse_dates=["timestamp_start","timestamp_end"])
        if bi.empty:
            print(f"[skip] empty {p_bi}")
            continue

        seg = detect_segments(bi, **SEG_PARAMS_SLOW)
        out = OUT_DIR / s / f"{tf}_segments.csv"
        seg.to_csv(out, index=False)
        print(f"[saved] {s}-{tf}: {len(seg)} 段 -> {out}")


[saved] BTCUSDT-5m: 43 段 -> /Users/cc/PycharmProjects/ChanLab/chan_data/segments/slow/BTCUSDT/5m_segments.csv
[saved] BTCUSDT-15m: 50 段 -> /Users/cc/PycharmProjects/ChanLab/chan_data/segments/slow/BTCUSDT/15m_segments.csv
[saved] BTCUSDT-30m: 37 段 -> /Users/cc/PycharmProjects/ChanLab/chan_data/segments/slow/BTCUSDT/30m_segments.csv
[saved] BTCUSDT-1h: 24 段 -> /Users/cc/PycharmProjects/ChanLab/chan_data/segments/slow/BTCUSDT/1h_segments.csv
[saved] BTCUSDT-4h: 3 段 -> /Users/cc/PycharmProjects/ChanLab/chan_data/segments/slow/BTCUSDT/4h_segments.csv
[saved] ETHUSDT-5m: 162 段 -> /Users/cc/PycharmProjects/ChanLab/chan_data/segments/slow/ETHUSDT/5m_segments.csv
[saved] ETHUSDT-15m: 94 段 -> /Users/cc/PycharmProjects/ChanLab/chan_data/segments/slow/ETHUSDT/15m_segments.csv
[saved] ETHUSDT-30m: 48 段 -> /Users/cc/PycharmProjects/ChanLab/chan_data/segments/slow/ETHUSDT/30m_segments.csv
[saved] ETHUSDT-1h: 27 段 -> /Users/cc/PycharmProjects/ChanLab/chan_data/segments/slow/ETHUSDT/1h_segments.csv
[s

In [22]:

from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
DATA_SEG = ROOT / "chan_data" / "segments" / "slow"
SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m","15m","30m","1h","4h"]

def pair_overlap_ratio(seg_df: pd.DataFrame) -> dict:
    if seg_df.empty:
        return dict(pairs=0, overlap_pairs=0, mean_overlap=0.0)
    df = seg_df.sort_values("segment_index")

    lows = df["low"].to_numpy(dtype=float)
    highs = df["high"].to_numpy(dtype=float)

    left = np.maximum(lows[:-1], lows[1:])
    right = np.minimum(highs[:-1], highs[1:])
    overlap = np.maximum(0.0, right - left)
    union = np.maximum(highs[:-1], highs[1:]) - np.minimum(lows[:-1], lows[1:])
    ratio = np.where(union > 0, overlap / union, 0.0)
    return dict(
        pairs=len(ratio),
        overlap_pairs=int((ratio > 0).sum()),
        mean_overlap=float(ratio.mean()) if len(ratio) else 0.0
    )

rows = []
for s in SYMBOLS:
    for tf in TFS:
        p = DATA_SEG / s / f"{tf}_segments.csv"
        if not p.exists():
            print(f"[skip] missing {p}")
            continue
        seg = pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])
        ov = pair_overlap_ratio(seg)
        rows.append(dict(
            symbol=s, tf=tf,
            total_segments=len(seg),
            pairs=ov["pairs"],
            overlap_pairs=ov["overlap_pairs"],
            overlap_ratio=round(ov["overlap_pairs"]/max(1,ov["pairs"]), 3),
            mean_overlap=round(ov["mean_overlap"], 3)
        ))
summary = pd.DataFrame(rows).sort_values(["symbol","tf"])
summary


symbol   tf  total_segments  pairs  overlap_pairs  overlap_ratio  \
1  BTCUSDT  15m              50     49             42          0.857   
3  BTCUSDT   1h              24     23             22          0.957   
2  BTCUSDT  30m              37     36             30          0.833   
4  BTCUSDT   4h               3      2              1          0.500   
0  BTCUSDT   5m              43     42             35          0.833   
6  ETHUSDT  15m              94     93             82          0.882   
8  ETHUSDT   1h              27     26             21          0.808   
7  ETHUSDT  30m              48     47             43          0.915   
9  ETHUSDT   4h               6      5              4          0.800   
5  ETHUSDT   5m             162    161            140          0.870   

   mean_overlap  
1         0.312  
3         0.354  
2         0.369  
4         0.114  
0         0.362  
6         0.334  
8         0.329  
7         0.322  
9         0.283  
5         0.355

In [24]:
from pathlib import Path
import pandas as pd
from chan import detect_segments

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
DATA_BI = ROOT / "chan_data" / "bi"
OUT_DIR = ROOT / "chan_data" / "segments" / "fast"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m","15m","30m","1h","4h"]


SEG_PARAMS_FAST = dict(
    min_bars=2,
    min_rel_move=0.001,
     enable_break_validation=False,
    break_mode="last_block_start",
    merge_same_dir=True,
    enable_quality_markers=True
)

def save_fast_segments(symbol: str, tf: str):
    bi_path = DATA_BI / symbol / f"{tf}_bi.csv"
    if not bi_path.exists():
        print(f"[skip] missing {bi_path}")
        return None

    bi = pd.read_csv(bi_path, parse_dates=["timestamp_start","timestamp_end"])
    if bi.empty:
        print(f"[skip] empty {bi_path}")
        return None

    seg = detect_segments(bi, **SEG_PARAMS_FAST)
    if seg.empty:
        print(f"[warn] no segments {symbol}-{tf}")
        return None

    out_dir = OUT_DIR / symbol
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{tf}_segments.csv"
    seg.to_csv(out_path, index=False)
    print(f"[saved-fast] {symbol}-{tf}: {len(seg)} 段 → {out_path}")
    return seg

for s in SYMBOLS:
    for tf in TFS:
        save_fast_segments(s, tf)


[saved-fast] BTCUSDT-5m: 112 段 → /Users/cc/PycharmProjects/ChanLab/chan_data/segments/fast/BTCUSDT/5m_segments.csv
[saved-fast] BTCUSDT-15m: 116 段 → /Users/cc/PycharmProjects/ChanLab/chan_data/segments/fast/BTCUSDT/15m_segments.csv
[saved-fast] BTCUSDT-30m: 105 段 → /Users/cc/PycharmProjects/ChanLab/chan_data/segments/fast/BTCUSDT/30m_segments.csv
[saved-fast] BTCUSDT-1h: 64 段 → /Users/cc/PycharmProjects/ChanLab/chan_data/segments/fast/BTCUSDT/1h_segments.csv
[saved-fast] BTCUSDT-4h: 17 段 → /Users/cc/PycharmProjects/ChanLab/chan_data/segments/fast/BTCUSDT/4h_segments.csv
[saved-fast] ETHUSDT-5m: 408 段 → /Users/cc/PycharmProjects/ChanLab/chan_data/segments/fast/ETHUSDT/5m_segments.csv
[saved-fast] ETHUSDT-15m: 256 段 → /Users/cc/PycharmProjects/ChanLab/chan_data/segments/fast/ETHUSDT/15m_segments.csv
[saved-fast] ETHUSDT-30m: 142 段 → /Users/cc/PycharmProjects/ChanLab/chan_data/segments/fast/ETHUSDT/30m_segments.csv
[saved-fast] ETHUSDT-1h: 78 段 → /Users/cc/PycharmProjects/ChanLab/chan_dat

In [2]:

from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path("/Users/cc/PycharmProjects/ChanLab")
SEG_DIR = ROOT / "chan_data" / "segments" / "fast"
SYMBOLS = ["BTCUSDT", "ETHUSDT"]
TFS = ["5m","15m","30m","1h","4h"]

def segment_overlap_stats(seg_df: pd.DataFrame):

    if len(seg_df) < 2:
        return dict(pairs=0, overlap_pairs=0, overlap_ratio=0.0, mean_overlap=0.0)

    seg = seg_df.sort_values("segment_index").reset_index(drop=True)
    lows  = seg["low"].to_numpy(dtype=float)
    highs = seg["high"].to_numpy(dtype=float)

    rng = np.maximum(highs - lows, 1e-12)

    lo_next = lows[1:]
    hi_next = highs[1:]
    lo_cur  = lows[:-1]
    hi_cur  = highs[:-1]

    inter_hi = np.minimum(hi_cur, hi_next)
    inter_lo = np.maximum(lo_cur, lo_next)
    inter_w  = np.maximum(0.0, inter_hi - inter_lo)


    norm = np.minimum(rng[:-1], rng[1:])
    strength = np.where(norm > 0, inter_w / norm, 0.0)

    pairs = len(seg) - 1
    overlap_pairs = int((inter_w > 0).sum())
    overlap_ratio = overlap_pairs / pairs if pairs else 0.0
    mean_overlap = float(strength[strength > 0].mean()) if (strength > 0).any() else 0.0

    return dict(pairs=pairs, overlap_pairs=overlap_pairs,
                overlap_ratio=overlap_ratio, mean_overlap=mean_overlap)

rows = []
for s in SYMBOLS:
    for tf in TFS:
        p = SEG_DIR / s / f"{tf}_segments.csv"
        if not p.exists():
            print(f"[skip] missing {p}")
            continue
        df = pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])
        if df.empty:
            print(f"[skip] empty {p}")
            continue
        ov = segment_overlap_stats(df)
        rows.append({
            "symbol": s,
            "tf": tf,
            "total_segments": len(df),
            "pairs": ov["pairs"],
            "overlap_pairs": ov["overlap_pairs"],
            "overlap_ratio": round(ov["overlap_ratio"], 3),
            "mean_overlap": round(ov["mean_overlap"], 3),
        })

fast_overlap_summary = pd.DataFrame(rows).sort_values(["symbol","tf"])
display(fast_overlap_summary)

print("\n== Gather ==")
print("total：", int(fast_overlap_summary["total_segments"].sum()))
print("Average overlap rate：", round(float(fast_overlap_summary["overlap_ratio"].mean()), 3))
print("Average overlap strength：", round(float(fast_overlap_summary["mean_overlap"].mean()), 3))

symbol   tf  total_segments  pairs  overlap_pairs  overlap_ratio  \
1  BTCUSDT  15m             116    115            110          0.957   
3  BTCUSDT   1h              64     63             62          0.984   
2  BTCUSDT  30m             105    104            100          0.962   
4  BTCUSDT   4h              17     16             14          0.875   
0  BTCUSDT   5m             112    111            103          0.928   
6  ETHUSDT  15m             256    255            250          0.980   
8  ETHUSDT   1h              78     77             74          0.961   
7  ETHUSDT  30m             142    141            139          0.986   
9  ETHUSDT   4h              20     19             19          1.000   
5  ETHUSDT   5m             408    407            394          0.968   

   mean_overlap  
1         0.804  
3         0.793  
2         0.832  
4         0.555  
0         0.754  
6         0.831  
8         0.832  
7         0.809  
9         0.811  
5         0.817


== Gather ==
total： 1318
Average overlap rate： 0.96
Average overlap strength： 0.784
